## Running this Jupyter Notebook will create two pdfs:
### * One named "upset_plots_group_b.pdf", containing 12 volcano plots. One for each knockout method per gene (3 * 4 = 12).
### * One named "volcano_plots_group_b.pdf", containing 4 upset plots showing the variation of DEGs between each knockout method per studied gene.

In [1]:
# Ensure that the BiocManager package is installed.
# BiocManager is required to install and manage Bioconductor packages.
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
# Install or update Bioconductor to version 3.18.
BiocManager::install(version = "3.18")
# Install data visualization package
BiocManager::install("ggplot2")
# Install differential gene expression analysis package
BiocManager::install("DESeq2")
# Install package for importing and exporting genomic data in various formats
BiocManager::install("rtracklayer")
# Install package for representing and manipulating genomic intervals
BiocManager::install("GenomicRanges")
# Install package for batch effect identification and correction.
BiocManager::install("sva")
# Install package for labelling volcano plots
install.packages("ggrepel")
# Install package for creating UpSet plots to better visualize the differences in each types DEGs
install.packages('UpSetR')

Bioconductor version '3.18' is out-of-date; the current release version '3.20'
  is available with R version '4.4'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.18 (BiocManager 1.30.25), R 4.3.3 (2024-02-29)

Old packages: 'askpass', 'backports', 'bit', 'bit64', 'bitops', 'broom',
  'bslib', 'caret', 'class', 'cli', 'clock', 'colorspace', 'commonmark',
  'cpp11', 'crayon', 'credentials', 'curl', 'data.table', 'DBI', 'dials',
  'downlit', 'e1071', 'evaluate', 'fontawesome', 'forecast', 'fs', 'future',
  'future.apply', 'gert', 'gower', 'gtable', 'hardhat', 'hexbin', 'highr',
  'httr2', 'ipred', 'jsonlite', 'KernSmooth', 'knitr', 'later', 'lava', 'lhs',
  'lubridate', 'modeldata', 'modelenv', 'nlme', 'nnet', 'openssl',
  'parallelly', 'parsnip', 'patchwork', 'pbdZMQ', 'pillar', 'p

In [2]:
# Load the required libraries for data analysis and visualization.
library(ggplot2)
library(DESeq2)
library(sva)
library(ggrepel)
library(UpSetR)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loa

#### Study 2- RNA-seq of male KOLF2.2J hiPSC-derived trophoblast cell lines homozygous null for seven different transcription factors. 
##### GEO Accession Number: GSE288289

In [3]:
# Read in raw count file
raw_cnts = read.csv("GSE288289_study2_genesRawCounts.csv",
                     header=TRUE,
                     sep="\t",
                     row.names=1,
                     strip.white=TRUE)
# Read in metadata file
meta_data = read.csv("metadata_study2.csv",
                      header=TRUE,
                      stringsAsFactors=FALSE)

#### Gene annotations generated using https://biit.cs.ut.ee/gprofiler/convert

        Liis Kolberg, Uku Raudvere, Ivan Kuzmin, Priit Adler, Jaak Vilo, Hedi Peterson: g:Profiler—interoperable web service for functional enrichment<br />
        analysis and gene identifier mapping (2023 update) Nucleic Acids Research, May 2023; doi:10.1093/nar/gkad347 [PDF]. <br />

In [4]:
# Load gene annotation data
gene_annotations = read.csv("gene_annotations.csv",
                      header=TRUE,
                      stringsAsFactors=FALSE)

#### Make necessary adjustments to match row/columns of meta data and counts and other data cleaning.

In [5]:
# Set row names for meta_data using the Sample column
# This ensures that each row in meta_data is indexed by its corresponding sample name
rownames(meta_data) = meta_data$Sample

# Standardize column names in raw_cnts:
# Remove trailing sequencing lane identifiers (_LXXX) from column names
colnames(raw_cnts) <- gsub("_L\\d{3}$", "", colnames(raw_cnts))
# Replace dots (.) with hyphens (-) in column names
colnames(raw_cnts) <- gsub("\\.", "-", colnames(raw_cnts))
# Remove leading and trailing whitespace from row names and the Gene column in meta_data
rownames(meta_data) <- trimws(meta_data$Sample)
meta_data$Gene <- trimws(meta_data$Gene)
# Verify that all row names in meta_data exist in the column names of raw_cnts
table(rownames(meta_data) %in% colnames(raw_cnts))
# Subset raw_cnts to include only columns that match the samples in meta_data
raw_cnts = raw_cnts[,rownames(meta_data)]
# Subset meta_data to include only columns used
meta_data <- subset(meta_data, select=-c(GEO.Accession, Scheme, Sample))


# Display the dimensions (rows x columns) of raw_cnts and meta_data
dim(raw_cnts)
dim(meta_data)

# Round raw counts to the nearest integer. 
# This is necessary for differential expression analysis, as DESeq2 requires integer counts
raw_cnts <- round(raw_cnts)


TRUE 
  82 

[1] 38592    82

[1] 82  2

#### Filter the dataset to focus specifically on the four genes of interest (MXD1, NC0A3, RUNX1, BMLHE40)

In [6]:
# Filter metadata to include only samples from specified gene types
# The "Type" column should be one of the following: "KO", "WT", "CE", or "PTC"
sample_counts <- table(meta_data$Gene[meta_data$Type %in% c("KO", "WT", "CE", "PTC")], 
                                meta_data$Type[meta_data$Type %in% c("KO", "WT", "CE", "PTC")])

# Identify genes with adequate representation across sample types
# A gene is considered valid if it has at least 3 samples per comparison group
valid_genes <- rownames(sample_counts)[apply(sample_counts, 1, function(x) all(x >= 3))]

# Define a list of genes of interest for analysis
selected_genes <- c("MXD1", "NCOA3", "RUNX1", "BHLHE40")

# Initialize lists to store results
results_list <- list()

#### <b>encode_types_as_factors:</b>  encode type vector in given meta_data as a factor, setting reference level to WT (Wild Type/Control) and return edited meta_data.

In [7]:
encode_types_as_factors <- function (meta_data) {
    meta_data$Type <- factor(meta_data$Type, levels=c("WT", "KO", "CE", "PTC"))
    meta_data$Type <- relevel(meta_data$Type, ref="WT")
    return(meta_data)
}

#### <b>filter_low_count_genes:</b>  return given counts without genes that dont have at least 10 counts in at least 3 samples filtered out.
    taken directly from DESeq2 vignette

In [8]:
filter_low_count_genes <- function (raw_cnts, smallestGroupSize = 3, minCounts = 10) {
    keep <- rowSums(raw_cnts >= minCounts) >= smallestGroupSize
    raw_cnts <- as.matrix(raw_cnts[keep, , drop = FALSE])
    return(raw_cnts)
}

#### <b>dataset_and_analysis:</b>  returns a DESeq data set with given counts and meta data, setting design to type (must be column in meta_data). Runs DESeq() analysis on dataset, which performs the following functions:
    estimation of size factors with estimateSizeFactors()
    estimation of dispersion with estimateDispersions()
    Negative Binomial GLM fitting and Wald statistics with nbinomWaldTest()

In [9]:
dataset_and_analysis <- function (raw_cnts, meta_data) {
    dds <- DESeqDataSetFromMatrix(countData = raw_cnts,
                                  colData = meta_data,
                                  design = ~ Type)
    dds <- DESeq(dds)
    return(dds)
}

#### <b>extract_results:</b>  return a list of result tables for each default contrast other than intercept.

In [10]:
extract_results <- function (dds) {
    return(list(
        KO = results(dds, name="Type_KO_vs_WT"),
        CE = results(dds, name="Type_CE_vs_WT"),
        PTC = results(dds, name="Type_PTC_vs_WT"))
    )
}

#### <b>Perform filtering, encoding, and analysis of each gene with at least 3 samples for each type, ensuring biological replicates for each type.</b>
#### - List of results for each gene are stored in a parent list, results_list, individually accessible by gene name.         
  i.e. results_list[["MXD1"]] 

In [ ]:
for (gene in selected_genes) {
    # select only samples of current gene
    filtered_meta_data <- meta_data[meta_data$Gene %in% c(gene), ]
    filtered_raw_cnts <- raw_cnts[, colnames(raw_cnts) %in% rownames(filtered_meta_data)]

    # prepare and filter data
    filtered_meta_data <- encode_types_as_factors(filtered_meta_data)
    filtered_raw_cnts <- filter_low_count_genes(filtered_raw_cnts)

    # create data set, run analysis, and store results 
    dds <- dataset_and_analysis(filtered_raw_cnts, filtered_meta_data)
    results_list[[gene]] <- extract_results(dds)
    
    # Print information about the filtered dataset for each gene
    cat("Gene:", gene, "\n")
    cat("Filtered metadata dimensions:", dim(filtered_meta_data), "\n")
    cat("Filtered raw counts dimensions:", dim(filtered_raw_cnts), "\n")
}

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates



#### <b>find_significant_genes:</b> given a DESeq results table, create dataframe with gene name annotations, and return list containing said dataframe, along with three other dataframes containing all significant DEGs, 

In [ ]:
# Parameters:
# - DESeq_results: A DESeq2 result object containing log2 fold changes and adjusted p-values
# - method: A string representing the method used (for labeling output)
# - l2fc_threshold: Log2 fold change threshold to classify significant genes (default = 0.5)
# - padj_threshold: Adjusted p-value threshold to classify significant genes (default = 0.05)
find_significant_genes <- function(DESeq_results, method, l2fc_threshold = 0.5, padj_threshold = 0.05) {
    
    # Convert DESeq2 results into a data frame and add gene names
    df <- as.data.frame(DESeq_results)
    df$gene <- rownames(df)

    # Merge with gene annotation data to obtain human-readable gene names
    df <- merge(df, gene_annotations[, c("initial_alias", "name")],
                by.x = "gene", by.y = "initial_alias", all.x = TRUE)
    
    # Identify significant differentially expressed genes (DEGs)
    # Genes are considered significant if:
    # - log2FoldChange > l2fc_threshold AND padj < padj_threshold (Upregulated)
    # - log2FoldChange < -l2fc_threshold AND padj < padj_threshold (Downregulated)
    significant_genes <- subset(df, ((log2FoldChange > l2fc_threshold & padj < padj_threshold) |
                                     (log2FoldChange < -l2fc_threshold & padj < padj_threshold)))

    # Sort significant genes:
    # - By log2 fold change (descending) to highlight most up/downregulated genes
    # - By adjusted p-value (ascending) to highlight most statistically significant genes
    sorted_by_l2fc_sig <- significant_genes[order(significant_genes$log2FoldChange, decreasing = TRUE), ]
    sorted_by_padj_sig <- significant_genes[order(significant_genes$padj), ]

    # Select genes to label on volcano plots:
    # - Top 5 most upregulated and 5 most downregulated genes by log2FoldChange
    # - Top 10 most statistically significant genes by padj
    labelled_genes <- c(sorted_by_l2fc_sig$name[1:5], sorted_by_l2fc_sig$name[(nrow(sorted_by_l2fc_sig) - 4):nrow(sorted_by_l2fc_sig)])
    labelled_genes <- unique(c(labelled_genes, sorted_by_padj_sig$name[1:10]))

    # Remove "None" labels (if any gene names are missing in annotations)
    labelled_genes <- labelled_genes[labelled_genes != "None"]
    
    # Add a new column to classify genes based on significance:
    df$Significant <- "NO"  # Default: Not significant
    df$Significant[df$log2FoldChange > l2fc_threshold & df$padj < padj_threshold] <- "UP"   # Upregulated genes
    df$Significant[df$log2FoldChange < -l2fc_threshold & df$padj < padj_threshold] <- "DOWN" # Downregulated genes
    
    # Store results in a list for easy retrieval and use in downstream analysis
    ret_list <- list(
        method = method,                # Method used for analysis
        res_df = df,                     # Complete results data frame
        significant_genes = significant_genes,  # Filtered DEGs
        sorted_by_l2fc = sorted_by_l2fc_sig,    # DEGs sorted by log2FC
        sorted_by_padj = sorted_by_padj_sig,    # DEGs sorted by adjusted p-value
        labelled_genes = labelled_genes        # Selected genes for volcano plot labeling
    )

    return(ret_list)
}


### Processing DESeq Results into Organized Data for Each Gene and Comparison

In [ ]:
names(results_list[["MXD1"]])

In [ ]:
processed_results_list <- list()

# Loop through each selected gene, processing DESeq results for every comparison type (KO, CE, PTC)
for (gene in selected_genes) {
  processed_results_list[[gene]] <- lapply(names(results_list[[gene]]), function(comparison_type) {
    find_significant_genes(
      DESeq_results = results_list[[gene]][[comparison_type]],
      method = paste(gene, comparison_type, "vs_WT", sep = "_")
    )
  })

  # Clearly label each comparison within the gene
  names(processed_results_list[[gene]]) <- names(results_list[[gene]])
}

# Example access:
head(processed_results_list[["NC0A3"]][["KO"]])


#### <b>volcano_plot_from_res_df:</b> creates a volcano plot to visualize differentially expressed genes (DEGs) from the DESeq2 results, highlighting all significant genes, and labelling genes with high log2FoldChange or adjusted p-values.

In [ ]:
# Parameters:
# - res_df: Data frame containing DESeq2 results, including log2FoldChange and adjusted p-values
# - labelled_genes: A character vector of genes to be specifically labeled in the plot
# - title: Title for the volcano plot
# - ylim: Maximum y-axis value (for scaling the plot)
# - xlim: Maximum x-axis value (for scaling the plot)
# - l2fc_threshold: Log2 fold change threshold for significance (default = 0.5)
# - padj_threshold: Adjusted p-value threshold for significance (default = 0.05)

volcano_plot_from_res_df <- function(res_df, labelled_genes, title, ylim, xlim, l2fc_threshold = 0.5, padj_threshold = 0.05) {
    
    # Subset data to include only genes that are specifically labeled in the plot
    labelled_data <- subset(res_df, name %in% labelled_genes)

    # Create a volcano plot using ggplot2
    plot <- ggplot(res_df, aes(x=log2FoldChange, y=-log10(padj), color=Significant, label=name)) +
        
        # Scatter plot of all genes, with transparency for better visualization
        geom_point(alpha=0.4) +
        
        # Assign colors: 
        #   - "DOWN" (significantly downregulated genes) in blue
        #   - "NO" (non-significant genes) in gray
        #   - "UP" (significantly upregulated genes) in red
        scale_color_manual(values=c("DOWN"="blue", "NO"="gray", "UP"="red")) +
        
        # Add vertical dashed lines to indicate log2FoldChange significance thresholds
        geom_vline(xintercept=c(-l2fc_threshold, l2fc_threshold), linetype="dashed", color="red") +
        
        # Add a horizontal dashed line to indicate the adjusted p-value threshold
        geom_hline(yintercept=-log10(padj_threshold), linetype="dashed", color="blue") +
        
        # Add gene labels for selected genes, avoiding overlapping labels
        geom_label_repel(aes(label = ifelse(name %in% labelled_genes, as.character(name), '')),
                    box.padding   = 1,    # Space around labels
                    point.padding = 0,    # Space between label and point
                    segment.color = 'grey50', # Line color connecting label to point
                    show_guide = FALSE) + # Hide guide
       
        # Set the plot title
        ggtitle(title) +

        # Set y-axis and x-axis limits
        ylim(0, ylim) + 
        xlim(-xlim, xlim)
}


### Generating and Storing Volcano Plots for Each Gene and Comparison

In [ ]:
# Set maximum label overlaps for ggrepel to improve label placement in volcano plots
options(ggrepel.max.overlaps = 100)

# Initialize an empty list to store volcano plots
plot_list <- list()

# Loop through each gene to generate volcano plots for all comparison types (KO, CE, PTC) vs WT
for (gene in names(processed_results_list)) {
  
  # Create a nested list for storing plots for each gene
  plot_list[[gene]] <- list()

  # Generate volcano plots for each specific comparison (KO, CE, PTC) vs WT
  for (comparison in names(processed_results_list[[gene]])) {
    
    # Extract processed DESeq2 result data for the current gene and comparison
    current_data <- processed_results_list[[gene]][[comparison]]
    
    # Create a title for the volcano plot
    plot_title <- paste(gene, comparison, "vs WT")

    # give space for labels for BHLHE40 gene
    if (gene == "BHLHE40") {
    x <- 4
    y <- 45
    } else {
    x <- 3
    y <- 35
    }
      
    # Generate the volcano plot and store it in the plot_list
    plot_list[[gene]][[comparison]] <- volcano_plot_from_res_df(
      res_df = current_data$res_df,         # Processed DESeq2 results data frame
      labelled_genes = current_data$labelled_genes,  # Selected significant genes for labeling
      title = plot_title,                   # Title of the plot
      xlim = x,                             # X-axis limits (log2FoldChange)
      ylim = y                             # Y-axis limits (-log10(padj))
    )
  }
}


In [ ]:
upset_plot_list <- list()
for (gene in names(processed_results_list)) {
    all_significant_genes <- unique(c(processed_results_list[[gene]][["KO"]][["significant_genes"]]$gene, 
                                  processed_results_list[[gene]][["CE"]][["significant_genes"]]$gene, 
                                  processed_results_list[[gene]][["PTC"]][["significant_genes"]]$gene))
    
    KO_set <- sapply(all_significant_genes, function(x) ifelse(x %in% processed_results_list[[gene]][["KO"]][["significant_genes"]]$gene, 1, 0))
    CE_set <- sapply(all_significant_genes, function(x) ifelse(x %in% processed_results_list[[gene]][["CE"]][["significant_genes"]]$gene, 1, 0))
    PTC_set <- sapply(all_significant_genes, function(x) ifelse(x %in% processed_results_list[[gene]][["PTC"]][["significant_genes"]]$gene, 1, 0))
    upset_df <- data.frame(all_significant_genes, KO_set, CE_set, PTC_set)
    # Set row names as gene names for easy reference
    rownames(upset_df) <- all_significant_genes
    
    # Remove the first column (gene names) to keep only the binary presence/absence data
    upset_df <- upset_df[,-1]
    colnames(upset_df) <- c("KO", "CE", "PTC")
    
    # Create the plot with the title indicating the gene
    upset_plot_list[[gene]] <- upset(upset_df,
                                    point.size = 2.5,
                                    line.size = 1.5,
                                    text.scale = 1.6,
                                    mainbar.y.label = paste(gene, "DEGs", sep = " ")
                                    )
}

In [ ]:
# Define the file path for saving the volcano plots as a PDF
# 'getwd()' retrieves the current working directory, and 'file.path()' ensures proper file path formatting
volcano_plots_path <- file.path(getwd(), "volcano_plots_group_b.pdf")

# Open a PDF device to save plots. All plots printed after this will be directed to this PDF file
pdf(volcano_plots_path)

# Loop through all genes and their respective comparison plots
for (gene in names(plot_list)) {
  for (comparison in names(plot_list[[gene]])) {
    
    # Print the volcano plot for the current gene and comparison
    # Since the PDF device is active, this will write the plot to the file instead of displaying it in Jupyter
    print(plot_list[[gene]][[comparison]])
  }
}

# Close the PDF device to finalize the file
dev.off()


In [ ]:
upset_plot_path <- file.path(getwd(), "upset_plots_group_b.pdf")

pdf(upset_plot_path, height = 8, width = 12)

upset_plot_list

dev.off()

## Batch Correction -
#### As the results when including batch effects in the design of the DESeq2 model were drastically different not only from our own plots, but from plots found on the MorPhiC data portal, we decided to not inlcude them in our final analysis.

#### The following is the workflow necessary for including surrogate variables in the design using the gene "BHLHE40" as an example. The workflow is practically one-to-one with the DESeq2-svaseq workflow vignette found here:

https://master.bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html#using-sva-with-deseq2

In [ ]:
# Uses last known value of dds, which would be the BHLHE40 dds as it's the last one done in the loop.
dat <- counts(dds, normalized = TRUE)
idx <- rowMeans(dat) > 1
dat <- dat[idx,]
mod <- model.matrix(~ Type, colData(dds))
mod0 <- model.matrix(~ 1, colData(dds))
svseq <- svaseq(dat, mod, mod0)
ddssva <- dds
ddssva$SV1 <- svseq$sv[,1]

In [ ]:
design(ddssva) <- ~ SV1 + Type
ddssva <- DESeq(ddssva)

# Now, results from the ddssva object will be batch corrected.

### Citations for packages used

In [ ]:
# List of packages
packages <- c("BiocManager", "ggplot2", "DESeq2", "rtracklayer", 
              "GenomicRanges", "sva", "ggrepel", "UpSetR")

# Get citations for each package
for (pkg in packages) {
  cat("\nCitation for", pkg, ":\n")
  print(citation(pkg))
  cat("\n--------------------------------------------\n")
}